In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install transformers
'''
# download Kaggle true fake news dataset
!pip install kaggle
import os
os.environ["KAGGLE_CONFIG_DIR"] = "/content/drive/MyDrive/Colab Notebooks/CS6120 NLP/Assignment 6 BERT"
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset
'''

from sklearn.model_selection import train_test_split
from sklearn import model_selection                   # K fold library
from torch.utils.data import DataLoader                 
from torch.optim import AdamW
import torch
import pandas as pd
import os
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, average_precision_score, average_precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer  
from sklearn import metrics 
import torch.nn.functional as F

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 73.7 MB/s eta 0:00:00


In [4]:
# Check GPU resources
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('Using GPU ', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('Using CPU')

Using CPU


In [5]:
# Prepare tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
print(tokenizer)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [6]:
# Model configuration Settings
#NUM_LABELS = 1
BATCH_SIZE = 32
MAX_LEN = 32
EPOCHS = 20

In [7]:
# Prepare dataset
path = "/content/drive/MyDrive/Colab Notebooks/CS6120 NLP/Assignment 6 BERT/fake-and-real-news-dataset/"
df_real = pd.read_csv(path + 'True.csv')
df_fake = pd.read_csv(path + 'Fake.csv')

# Add y_true
df_real['Category'] = 1
df_fake['Category'] = 0

# Combine true news and fake news into one single file
df = df_real.append(df_fake)

<ipython-input-7-21ddf76171d3>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df_real.append(df_fake)


In [8]:
# Diminish dataset to 25% due to training time
dataset_75, dataset_25 = train_test_split(df, test_size=0.01, shuffle=True)
df = dataset_25

In [9]:
from torch.utils.data import Dataset

class NewsDataset(Dataset):
  def __init__(self, train_kf, tokenizer, max_len):
    self.train_kf = train_kf
    self.tokenizer = tokenizer
    self.max_len = max_len
    

  def __getitem__(self, index):
    # convert news docment to input ids using BERT tokenizer
    text = self.train_kf.iloc[index]["text"]
    category = self.train_kf.iloc[index]["Category"]

    tokenizer_dict = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=self.max_len)
    input_ids = tokenizer_dict['input_ids']
    attention_mask = tokenizer_dict['attention_mask'] 
    y = torch.tensor(category)

    #print(input_ids, attention_mask, y)
    #print(input_ids.shape, attention_mask.shape, y.shape)

    return input_ids, attention_mask, y

  def __len__(self):
    return len(self.train_kf)

In [10]:
def train(train_dataloader, model, device, lr=2e-5, warmup_steps=200):
    model.to(device)
    model.train()
    
    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1)
    batch_loss = 0
  
    for batch in train_dataloader:
      input_ids, attention_mask, labels = tuple(item.to(device) for item in batch)
      
      # without these 3 statements, despite tensor shape [batch size, seq_len] is correctly output, 
      # at here it still in [batch size, 1, seq_len] shape, so need to reshape here
      input_ids = input_ids.squeeze(1)
      attention_mask = attention_mask.squeeze(1)
      labels = labels.unsqueeze(0)

      optimizer.zero_grad() 

      # forward pass, outputs object contains the model's predictions, as well as the loss and other optional outputs.
      outputs = model(input_ids, attention_mask, labels=labels)  

      # default loss function by huggingface BERT - binary_cls: nll, multiclass: cross entropy
      # extracts the loss value from the outputs object
      loss = outputs.loss
      batch_loss += outputs.loss        # tensor in format 


      loss.backward() 
      optimizer.step() 
      scheduler.step()
    
    return batch_loss/len(train_dataloader)

In [11]:
def validate(val_dataloader, model, device):
    model.to(device)
    model.eval()

    total_loss, total_acc = 0, 0
    y_val = []
    labels_list = []

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = tuple(item.to(device) for item in batch)

            input_ids = input_ids.squeeze(1)
            attention_mask = attention_mask.squeeze(1)
            labels = labels.unsqueeze(0)

            # outputs produces 2 parameters: outputs.loss, outputs.logits
            outputs = model(input_ids, attention_mask, labels=labels) 

            # store prediction
            # outputs.logits represents the model's confidence scores for each class label
            y_val.append(outputs.logits.argmax(axis=1).tolist())
            labels_list.append(labels.tolist())

            #print("outputs.logits.argmax(axis=1)\n", outputs.logits.argmax(axis=1))

            y_val_flatten = [item for sublist in y_val for item in sublist]
            labels_flatten = [item for sublist1 in labels_list for sublist2 in sublist1 for item in sublist2]

    print("y_val_flatten\n", y_val_flatten)  
    print("labels_flatten\n", labels_flatten)
    print(classification_report(y_val_flatten, labels_flatten))
        

In [ ]:
# Define learning rates to test
learning_rates = [0.0001, 0.001, 0.01]
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=42)
best_lr = None
best_avg_loss = float('inf')

for fold, (train_index, valid_index) in enumerate(kf.split(df)):    # kf.split() method returns an iterator that generates a set of train and validation indices for each fold.

    # Get the training and validation data for this fold
    train_kf = df.iloc[train_index]
    val_kf = df.iloc[valid_index]

    print("fold", fold)
    print(len(train_kf))  # 5 fold is 80%
    print(len(val_kf))   # 5 fold is 20%

    # Get tokens and attention mask
    train_dataset = NewsDataset(train_kf, tokenizer, max_len=MAX_LEN)
    val_dataset = NewsDataset(val_kf, tokenizer, max_len=MAX_LEN)

    # Get dataloader
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
    val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

    early_stopping_patience = 4

    for lr in learning_rates:
        print("lr", lr)

        model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
        previous_loss = 100

        for epoch in range(EPOCHS):

            # Train the model on this fold with the current learning rate
            loss = train(train_dataloader, model, device, lr=lr, warmup_steps=200)
            print("Epoch: {}, loss: {}".format(epoch, loss))

            if loss < previous_loss:
                previous_loss = loss
                num_epochs_no_improvement = 0
            else:
                num_epochs_no_improvement += 1
                if num_epochs_no_improvement == early_stopping_patience:
                    print(f"Validation loss has not improved for {early_stopping_patience} epochs. Stopping training.")
                    break

fold 0
359
90
lr 0.0001


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch: 0, loss: 0.6687213778495789


In [ ]:
train_df, val_df = train_test_split(df, test_size=0.25, shuffle=True)

train_dataset = NewsDataset(train_df, tokenizer, max_len=MAX_LEN)
val_dataset = NewsDataset(val_df, tokenizer, max_len=MAX_LEN)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

lr = 0.01

for epoch in range(EPOCHS):

    loss = train(train_dataloader, model, device, lr=lr, warmup_steps=200)
    print("Epoch: {}, loss: {}".format(epoch, loss))

    if loss < previous_loss:
        previous_loss = loss
        num_epochs_no_improvement = 0
    else:
        num_epochs_no_improvement += 1
        if num_epochs_no_improvement == early_stopping_patience:
            print(f"Validation loss has not improved for {early_stopping_patience} epochs. Stopping training.")
            break

validate(val_dataloader, model, device)


In [ ]:
#!tensorboard dev upload --logdir "/content/drive/MyDrive/Colab Notebooks/CS6120 NLP/CS6120 Project 6" --name "image captioning 6.5 epochs"
